<a href="https://colab.research.google.com/github/Maxie503/PWSKILL-Learnings/blob/main/Class_326_Resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip '/content/cat-dog-data.zip'

Archive:  /content/cat-dog-data.zip
   creating: train/
   creating: train/Cat/
  inflating: train/Cat/0.jpg         
  inflating: train/Cat/1.jpg         
  inflating: train/Cat/2.jpg         
  inflating: train/Cat/cat.2405.jpg  
  inflating: train/Cat/cat.2406.jpg  
  inflating: train/Cat/cat.2436.jpg  
  inflating: train/Cat/cat.2437.jpg  
  inflating: train/Cat/cat.2438.jpg  
  inflating: train/Cat/cat.2439.jpg  
  inflating: train/Cat/cat.2440.jpg  
  inflating: train/Cat/cat.2441.jpg  
  inflating: train/Cat/cat.2442.jpg  
  inflating: train/Cat/cat.2443.jpg  
  inflating: train/Cat/cat.2444.jpg  
  inflating: train/Cat/cat.2445.jpg  
  inflating: train/Cat/cat.2446.jpg  
  inflating: train/Cat/cat.2447.jpg  
  inflating: train/Cat/cat.2448.jpg  
  inflating: train/Cat/cat.2449.jpg  
  inflating: train/Cat/cat.2450.jpg  
  inflating: train/Cat/cat.2451.jpg  
  inflating: train/Cat/cat.2452.jpg  
  inflating: train/Cat/cat.2453.jpg  
  inflating: train/Cat/cat.2454.jpg  
  inflat

In [50]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, AveragePooling2D, Flatten, Dense, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

In [40]:
from keras.utils import to_categorical

In [41]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Found 337 images belonging to 2 classes.
Found 59 images belonging to 2 classes.


In [71]:
model = resnet(shape=(224, 224, 3), num_classes=2)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data generators with categorical class_mode for one-hot encoding
train_gen = ImageDataGenerator(rescale=1./255)
valid_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(directory='/content/train',
                                           target_size=(224, 224),
                                           batch_size=32,
                                           class_mode='categorical')

vali_data = valid_gen.flow_from_directory(directory='/content/validation',
                                           target_size=(224, 224),
                                           batch_size=32,
                                           class_mode='categorical')

# Train the model
model.fit(train_data, epochs=5, verbose=1, validation_data=vali_data)

Found 337 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Epoch 1/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 180s 15s/step - accuracy: 0.4809 - loss: 22.1844 - val_accuracy: 0.5254 - val_loss: 1.3871
Epoch 2/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 159s 14s/step - accuracy: 0.4761 - loss: 5.0804 - val_accuracy: 0.4746 - val_loss: 1.5956
Epoch 3/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 199s 14s/step - accuracy: 0.5506 - loss: 2.6107 - val_accuracy: 0.4915 - val_loss: 0.9408
Epoch 4/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 157s 14s/step - accuracy: 0.5676 - loss: 1.0168 - val_accuracy: 0.5085 - val_loss: 0.6928
Epoch 5/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 155s 14s/step - accuracy: 0.6469 - loss: 0.8479 - val_accuracy: 0.4915 - val_loss: 0.7339



train_gen = ImageDataGenerator(rescale=1./255)

valid_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(directory='/content/train',
                                    target_size=(224,224),
                                    batch_size=32, class_mode='binary')

vali_data = valid_gen.flow_from_directory(directory='/content/validation',
                                    target_size=(224,224),
                                    batch_size=32, class_mode='binary')

def residual_block(x, filters, downsample=False):
    strides = (2,2) if downsample else (1,1)

    # Conv1
    y = Conv2D(filters, kernel_size=(3,3), strides=strides, padding='same')(x)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    # Skip connection: Always match channels
    if x.shape[-1] != filters or downsample:
        x = Conv2D(filters, kernel_size=(1,1), strides=strides, padding='same')(x)

    # Add residual
    y = Add()([x, y])
    y = Activation('relu')(y)
    return y


def resnet(shape, num_classes):
    inputs = keras.Input(shape=shape)

    x = Conv2D(64, kernel_size=(3,3), strides=(2,2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Residual Blocks
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    x = residual_block(x, 128, True)
    x = residual_block(x, 128)
    x = residual_block(x, 256, True)
    x = residual_block(x, 256)

    # Final layers
    x = AveragePooling2D(pool_size=(2,2))(x)
    x = Flatten()(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=output)
    return model


# Create and compile model
model = resnet(shape=(224, 224, 3), num_classes=2)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data, epochs=5, verbose=1)